In [1]:
from pyomo.environ import *
import pandas as pd
import math
#https://app.code2flow.com/BTjnFY


  $$c = \sqrt{a^2 + b^2}$$


In [2]:
from IPython.display import display, Math, Latex
display(Math(r'F(k) = \int_{-\infty}^{\infty} f(x) e^{2\pi i k} dx'))


<IPython.core.display.Math object>

  ## Major

  1.  Use TimeSeries of solar and wind in the constraint (impossible)
  2.  Match the result with Mamas sheet (yes)
  3.  Use panda profiling to generate all htmls (yes)
  4.  Show all equations in latex (later)
  5.  Make optimization more general (yes)
  6.  get euro_process working (yes)
  6.  get constants from excel file 
  7. Clean up code (yes)
  8. Make flowchart (yes)
  9. energy, renewable energies, clean tech and env policy compare publishers and impact factors (yes)
  10. clean dead files (yes)
  11. Make website (yes)

  ### Minor
  1. Rename TimeSeriesProcess to ts_process

  # Read and Format

In [3]:
processed_entire_dict_df = pd.read_excel("input/new_format-2020.xlsx", sheet_name=None)
region_processes=list(processed_entire_dict_df["SupIm"].columns.values)[1:]

def add_capacity_col_to_SupIm(processed_entire_dict_df):


    def fill_up_capacity(row, col_name):

        if ("Wind" in col_name):

            if 4 < row[col_name] <= 25:
                cell = row[col_name]

                return (math.exp(-((4/12)**2.25))-math.exp(-((cell/12)**2.25)))/((cell/12)**2.25-(4/12)**2.25)-(math.exp(-((25/12)**2.25)))
            else:
                return 0

        else:
            cell = row[col_name]
            if cell == 0:
                return 0
            else:
                return 0.025+(0.21-0.025)/((processed_entire_dict_df["SupIm"])[col_name].max()-1)*(cell-1)


    for i in range(len(region_processes)):
        norm_name = region_processes[i]
        processed_entire_dict_df["SupIm"][norm_name+".Capacity"] = processed_entire_dict_df["SupIm"].apply(
            lambda x: fill_up_capacity(x, norm_name),  axis=1)


add_capacity_col_to_SupIm(processed_entire_dict_df)



In [4]:
def add_unit_cost_col_to_Process(processed_entire_dict_df):

    df = processed_entire_dict_df["ProcessData"]

    df['Unit Cost (Euro/MW)'] = df["Variable Cost (Euro/MW/a)"] +df["Investment Cost (Euro/MW)"]*(df["WACC"] * (1+df["WACC"])**df["Life  (years)"])/(
        (1+df["WACC"])**df["Life  (years)"]-1) + df["Fixed Cost (Euro/MW/a)"]


add_unit_cost_col_to_Process(processed_entire_dict_df)


def add_unit_cost_col_to_Storage(processed_entire_dict_df):
    
    df = processed_entire_dict_df["StorageData"]

    df['Unit Cost (Euro/MW)'] = df["Variable Cost (Euro/MW/a)"] +df["Investment Cost (Euro/MWh)"]*(df["WACC"] * (1+df["WACC"])**df["Life  (years)"])/(
        (1+df["WACC"])**df["Life  (years)"]-1) + df["Fixed Cost (Euro/MWh/a)"]


add_unit_cost_col_to_Storage(processed_entire_dict_df)


In [5]:

def add_mw_col_to_Process(processed_entire_dict_df):

    SupIm_df = processed_entire_dict_df["SupIm"]
    for i in range(len(region_processes)):
        norm_name = region_processes[i]

        if ("Wind" in norm_name):

            SupIm_df[norm_name+".CellPower"] = 0.6*8000 * \
                SupIm_df[norm_name+".Capacity"] * \
                SupIm_df[norm_name]**3/10**6

            SupIm_df[norm_name+".CapacityFactorRatio"] = SupIm_df[norm_name +
                                                                        ".CellPower"]/SupIm_df[norm_name+".CellPower"].mean()
        else:
            SupIm_df[norm_name+".CapacityFactorRatio"] = SupIm_df[norm_name +
                                                                        ".Capacity"] / SupIm_df[norm_name + ".Capacity"].mean()


add_mw_col_to_Process(processed_entire_dict_df)

processed_entire_dict_df.keys(
    
)

processed_entire_dict_df["ResourceCost"]

,Resource,Cost (Euro/MWh)
0,Wind,0
1,Solar,0
2,Hydropower,0
3,Geothermal,0
4,Coal plant,27
5,Gas plant,7
6,Biomass plant,6


  ## Sets

In [6]:
# CONSTS
HOUR_TO_SECONDS=60*60
TONNE_TO_GRAM=10**6
CO2_PRICE=500
TIME_SERIES_PROCESSES = list(processed_entire_dict_df["TSProcess"]["Process"])
RENEWABLE_PROCESSES=list(processed_entire_dict_df["ResourceCost"].loc[processed_entire_dict_df["ResourceCost"]['Cost (Euro/MWh)'] == 0]["Resource"])
# Sets

Regions = {x.replace('.Elec', '')
           for x in list(processed_entire_dict_df["Demand"].columns.values)[1:]}
Processes = list(processed_entire_dict_df["ProcessData"]['Process'].unique())
Time = list(processed_entire_dict_df["SupIm"]['t'])

# Parameters
CommodityPrices = pd.Series(
    processed_entire_dict_df["ResourceCost"]["Cost (Euro/MWh)"].values, index=processed_entire_dict_df["ResourceCost"]["Resource"].values).to_dict()
ProcessUnitCost = pd.Series(processed_entire_dict_df["ProcessData"]["Unit Cost (Euro/MW)"].values, index=(
    processed_entire_dict_df["ProcessData"].Process.values)).to_dict()
StorageEfficiency=dict(zip(processed_entire_dict_df["StorageData"].Storage.values, processed_entire_dict_df["StorageData"].Efficiency.values))  
StorageUnitCost = pd.Series(processed_entire_dict_df["StorageData"]["Unit Cost (Euro/MW)"].values, index=(
    processed_entire_dict_df["StorageData"].Storage.values)).to_dict()    
EfficiencyRatio = pd.Series(processed_entire_dict_df["ProcessData"]["Efficiency"].values, index=(
    processed_entire_dict_df["ProcessData"].Process.values)).to_dict()
ProcessCapUp = pd.Series(processed_entire_dict_df["ProcessData"]["Maximum Capacity (MW)"].values,
                         index=(processed_entire_dict_df["ProcessData"].Process.values)).to_dict()
GWPProcess = pd.Series(processed_entire_dict_df["ProcessData"]["GWP (g/MJ)"].values, index=(
    processed_entire_dict_df["ProcessData"].Process.values)).to_dict()

# Sets
model = ConcreteModel()
model.Regions = Set(initialize=list(Regions))
model.Processes = Set(initialize=list(Processes))
model.Time = Set(initialize=list(Time))
model.TIME_SERIES_PROCESSES = Set(initialize=list(TIME_SERIES_PROCESSES))
StorageEfficiency


{'Battery': 0.91}

  # Inits

In [7]:


def commoditiy_prices_init(model, process):
    return CommodityPrices[process]


def process_unit_cost_init(model, process):
    return ProcessUnitCost[process]

def storage_unit_cost_init(model):
    return StorageUnitCost["Battery"]

def efficiency_ratio_init(model, process):
    return EfficiencyRatio[process]

def storage_efficiency_init(model):
    return StorageEfficiency["Battery"]


def process_cap_up_init(model, process):
    return ProcessCapUp[process]

def gwp_process_init(model, process):
    return GWPProcess[process]

def average_demand_region_init(model, region):
    col = region
    return processed_entire_dict_df["Demand"][col].mean()


def supim_capacity_factor_ratio_init(model, time, regions, process):

    col = regions+"."+process+".CapacityFactorRatio"
    row = int(float(time))
    return processed_entire_dict_df["SupIm"].at[row, col]


def demand_region_init(model, time, regions):

    col = regions
    row = int(float(time))
    return processed_entire_dict_df["Demand"].at[row, col]

processed_entire_dict_df["SupIm"]

,t,SouthWest.Wind,SouthWest.Solar,SouthEast.Wind,SouthEast.Solar,GreaterLondon.Wind,GreaterLondon.Solar,SouthWest.Wind.Capacity,SouthWest.Solar.Capacity,SouthEast.Wind.Capacity,...,GreaterLondon.Solar.Capacity,SouthWest.Wind.CellPower,SouthWest.Wind.CapacityFactorRatio,SouthWest.Solar.CapacityFactorRatio,SouthEast.Wind.CellPower,SouthEast.Wind.CapacityFactorRatio,SouthEast.Solar.CapacityFactorRatio,GreaterLondon.Wind.CellPower,GreaterLondon.Wind.CapacityFactorRatio,GreaterLondon.Solar.CapacityFactorRatio
0,0,6.471513,0.0,6.606024,0.0,8.000000,0.0,0.841865,0.0,0.837018,...,0.0,1.095219,0.423663,0.0,1.158235,0.500326,0.0,1.922126,2.225635,0.0
1,1,7.327003,0.0,7.330003,0.0,5.333333,0.0,0.809627,0.0,0.809509,...,0.0,1.528641,0.591323,0.0,1.530295,0.661046,0.0,0.640194,0.741282,0.0
2,2,8.556605,0.0,8.178551,0.0,6.333333,0.0,0.758152,0.0,0.774538,...,0.0,2.279828,0.881904,0.0,2.033822,0.878557,0.0,1.032510,1.195546,0.0
3,3,7.579646,0.0,9.040541,0.0,5.333333,0.0,0.799503,0.0,0.736573,...,0.0,1.671123,0.646439,0.0,2.612403,1.128488,0.0,0.640194,0.741282,0.0
4,4,7.816800,0.0,8.191993,0.0,5.333333,0.0,0.789773,0.0,0.773963,...,0.0,1.810634,0.700406,0.0,2.042350,0.882240,0.0,0.640194,0.741282,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,8755,17.370674,0.0,14.027099,0.0,9.666667,0.0,0.364311,0.0,0.501554,...,0.0,9.165664,3.545546,0.0,6.644503,2.870247,0.0,3.068998,3.553602,0.0
8756,8756,17.697677,0.0,16.101136,0.0,8.666667,0.0,0.352635,0.0,0.412755,...,0.0,9.382442,3.629402,0.0,8.269959,3.572401,0.0,2.353780,2.725449,0.0
8757,8757,16.061313,0.0,20.081081,0.0,12.000000,0.0,0.414353,0.0,0.277625,...,0.0,8.240517,3.187672,0.0,10.790966,4.661408,0.0,4.947989,5.729291,0.0
8758,8758,15.209872,0.0,17.633122,0.0,16.666667,0.0,0.449570,0.0,0.354914,...,0.0,7.593040,2.937209,0.0,9.340107,4.034676,0.0,8.679407,10.049911,0.0


In [8]:
model.commoditiy_prices = Param(
    model.Processes, initialize=commoditiy_prices_init)
model.process_unit_cost = Param(
    model.Processes, initialize=process_unit_cost_init)

model.storage_unit_cost=Param(initialize=storage_unit_cost_init)
model.efficiency_ratio = Param(
    model.Processes, initialize=efficiency_ratio_init)
model.storage_efficiency=Param(initialize=storage_efficiency_init)
model.process_cap_up = Param(model.Processes, initialize=process_cap_up_init)
model.average_demand_region = Param(
    model.Regions, rule=average_demand_region_init)
model.supim_capacity_factor_ratio = Param(model.Time, model.Regions, model.TIME_SERIES_PROCESSES,
                                          initialize=supim_capacity_factor_ratio_init)
# not a var
model.demand_region = Param(
    model.Time, model.Regions, initialize=demand_region_init, doc="fdsf")
model.process_adjust = Param(
    model.Regions, model.Processes, domain=NonNegativeReals, initialize=1)

model.gwp_process=Param(model.Processes,initialize=gwp_process_init)


# Variables
model.region_fraction = Var(
    model.Regions, model.Processes, domain=NonNegativeReals, initialize=0.01)



  # Constraints

In [9]:


def region_fraction_sum_rule(model, region):

    s = 0.0
    for process in model.Processes:
        s += model.region_fraction[region, process]

    return s == 1


def demand_region_lt_cap_up_rule(model, region, process):
    return model.region_fraction[region, process]*model.average_demand_region[region] <= model.process_cap_up[process]



def deficit_sum_rule(model,region):
    deficit_sum=0

    for time in model.Time:
        deficit_sum +=model.demand_region[time,region]
        for process in model.Processes:
            if process not in TIME_SERIES_PROCESSES: 
            
                deficit_sum +=- model.region_fraction[region, process]*model.average_demand_region[region]
            else:
                deficit_sum +=-model.region_fraction[region,process]*model.average_demand_region[region]*model.supim_capacity_factor_ratio[time,region,process]

        # deficit_sum+=model.demand_region[time,region] -model.region_fraction[region, "Solar"]*model.average_demand_region[region]*model.supim_capacity_factor_ratio[time,region,"Solar"]- model.region_fraction[region, "Wind"]*model.average_demand_region[region] *  model.supim_capacity_factor_ratio[time,region, "Wind"] - model.region_fraction[region, "Biomass plant"]*model.average_demand_region[region]- model.region_fraction[region, "Coal plant"]*model.average_demand_region[region]- model.region_fraction[region, "Gas plant"]*model.average_demand_region[region]
    return deficit_sum <=0




In [10]:
model.region_fraction_sum = Constraint(
    model.Regions, rule=region_fraction_sum_rule)
model.demand_region_less_than_cap_up = Constraint(
    model.Regions, model.Processes, rule=demand_region_lt_cap_up_rule)
model.deficit_sum = Constraint(model.Regions, rule = deficit_sum_rule)


  # Modelling

In [11]:
region_max = {}

print(value(model.storage_efficiency))
for region in model.Regions:

    max1 = 0

    for time in model.Time:

        expr=0

        expr +=model.demand_region[time,region]
        for process in model.Processes:
            if process not in TIME_SERIES_PROCESSES:

                expr +=- value(model.region_fraction[region, process])*model.average_demand_region[region]
            else:
                expr +=-value(model.region_fraction[region,process])*model.average_demand_region[region]*model.supim_capacity_factor_ratio[time,region,process]

        # expr = model.demand_region[time, region] - value(model.region_fraction[region, "Solar"])*model.average_demand_region[region]*model.supim_capacity_factor_ratio[time, region, "Solar"] - value(model.region_fraction[region, "Wind"])*model.average_demand_region[region] * model.supim_capacity_factor_ratio[time, region, "Wind"] - value(
        #     model.region_fraction[region, "Biomass plant"])*model.average_demand_region[region] - value(model.region_fraction[region, "Coal plant"])*model.average_demand_region[region] - value(model.region_fraction[region, "Gas plant"])*model.average_demand_region[region]

        if (max1 < expr):

            max1 = expr
    region_max[region] = max1


def ObjRule(model):
    tot = 0
    for region in model.Regions:
        for process in model.Processes:

            sum_of_renewables=sum( value(model.region_fraction[region, process])for region in model.Regions for process in model.Processes if process in RENEWABLE_PROCESSES )
            if (sum_of_renewables>=0.2*len(model.Regions)):
                tot += model.region_fraction[region, process] * model.average_demand_region[region] * (model.process_unit_cost[process] + (model.commoditiy_prices[process]+model.gwp_process[process]*HOUR_TO_SECONDS/TONNE_TO_GRAM*CO2_PRICE)*len(
                model.Time))/model.efficiency_ratio[process] * model.process_adjust[region, process] + (model.storage_unit_cost/model.storage_efficiency)*region_max[region]*0.8

            else:

                tot+=model.region_fraction[region, process] *model.average_demand_region[region] *(model.process_unit_cost[process] +model.commoditiy_prices[process]*len(model.Time)/model.efficiency_ratio[process]) *model.process_adjust[region, process]
    return tot
    # return sum(model.region_fraction[region, process] *
    #            model.average_demand_region[region] *
    #            (model.process_unit_cost[process] +
    #             model.commoditiy_prices[process]*len(model.Time)/model.efficiency_ratio[process]) *
    #            model.process_adjust[region, process] + (model.storage_unit_cost/model.storage_efficiency)*region_max[region]*0.8 for process in model.Processes for region in model.Regions)
    # return tot


model.obj = Objective(rule=ObjRule, sense=minimize)
# a=sum((model.storage_unit_cost/model.storage_efficiency)*region_max[region]*0.8 for region in model.Regions)
# b=sum ( value(model.region_fraction[region, process]) *model.average_demand_region[region] *(model.process_unit_cost[process] +model.commoditiy_prices[process]*len(model.Time)/model.efficiency_ratio[process]) *model.process_adjust[region, process] for process in model.Processes for region in model.Regions)
# b
model.obj.display()
sum_of_renewables = sum(value(model.region_fraction[region, process])
                        for region in model.Regions for process in model.Processes if process in RENEWABLE_PROCESSES)
sum_of_renewables
len(model.Regions)
sum_of_renewables >= 0.2*len(model.Regions)
print(sum_of_renewables, sum_of_renewables >= 0.2*len(model.Regions))

model.display()


0.91
obj : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 202267764.63652146
0.11999999999999998 False
Model unknown

  Variables:
    region_fraction : Size=21, Index=region_fraction_index
        Key                                : Lower : Value : Upper : Fixed : Stale : Domain
        ('GreaterLondon', 'Biomass plant') :     0 :  0.01 :  None : False : False : NonNegativeReals
           ('GreaterLondon', 'Coal plant') :     0 :  0.01 :  None : False : False : NonNegativeReals
            ('GreaterLondon', 'Gas plant') :     0 :  0.01 :  None : False : False : NonNegativeReals
           ('GreaterLondon', 'Geothermal') :     0 :  0.01 :  None : False : False : NonNegativeReals
           ('GreaterLondon', 'Hydropower') :     0 :  0.01 :  None : False : False : NonNegativeReals
                ('GreaterLondon', 'Solar') :     0 :  0.01 :  None : False : False : NonNegativeReals
                 ('GreaterLondon', 'Wind') :     0 :  0.01 :  None : False 

In [12]:
#opt = SolverFactory("glpk")
opt = SolverFactory("ipopt")
# ipopt is non linear
results = opt.solve(model,logfile = "name.csv")
print("\nDisplaying Solution\n" + '-'*60)


model.region_fraction.display()
# model.obj.display()
# # model.pprint() 
# # assert value(model.obj) ==812004985.3500707 , "Ur objective is bad"
# with open("something.txt", "w") as f:
#     for var in model.component_data_objects(Var):
#         f.write("%s: %s\n" % (var, var.value))
# model.region_fraction .display()



    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.11.1\x3a Converged to a locally
        infeasible point. Problem may be infeasible.

Displaying Solution
------------------------------------------------------------
region_fraction : Size=21, Index=region_fraction_index
    Key                                : Lower : Value                : Upper : Fixed : Stale : Domain
    ('GreaterLondon', 'Biomass plant') :     0 :  0.00998038074728411 :  None : False : False : NonNegativeReals
       ('GreaterLondon', 'Coal plant') :     0 : 0.009089804382427083 :  None : False : False : NonNegativeReals
        ('GreaterLondon', 'Gas plant') :     0 : 0.017075624108892375 :  None : False : False : NonNegativeReals
       ('GreaterLondon', 'Geothermal') :     0 : 0.005105354872545069 :  None : False : False : NonNegativeReals
       ('GreaterLondon', 'Hydropower') :     0 : 0.006060658295158632 :  None : False : False : NonNegativeReals
     